In [57]:
!pip install awswrangler==3.12.0 evidently==0.7.8 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
dash 2.18.1 requires Flask<3.1,>=1.0.4, but you have flask 3.1.0 which is incompatible.
dash 2.18.1 requires Werkzeug<3.1, but you have werkzeug 3.1.3 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.


In [78]:
import boto3
from botocore.exceptions import ClientError
from decimal import Decimal
import pandas as pd
import awswrangler as wr
from sklearn.metrics import log_loss, roc_auc_score

# READING LOGGING

In [51]:
LOG_TABLE = "mlops-utec-table-fraud-faas-log-rickpuma" # User your own table

In [52]:
dynamodb_read = boto3.client('dynamodb')

In [66]:
response = dynamodb_read.scan(TableName=LOG_TABLE)

In [67]:
def flatten_item(item):
    return {k: list(v.values())[0] for k, v in item.items()}

In [68]:
flat_items = [flatten_item(item) for item in response['Items']]
# Convert to DataFrame
df_pred = pd.DataFrame(flat_items)

In [69]:
df_pred.head()

,trx_vel_last_2mths,amt_vel_last_1mths,card_present,cod_month,trx_vel_last_1mths,transaction_id,prediction,amt_vel_last_2mths
0,10,640.6695,1,202501,4,65930a764da2,0.0001422771019861102,1391.082
1,7,450.68726,1,202501,3,65faa84a083c,0.0001422771019861102,983.5444


# READING FEEDBACK

In [61]:
query = """
    SELECT  transaction_id
            ,is_fraud
    FROM    RISK_MANAGEMENT.CREDIT_CARD_FEEDBACK
"""

In [70]:
df_feedback = wr.athena.read_sql_query(sql=query, database="risk_management")

In [72]:
df_report = df_pred.merge(df_feedback, on='transaction_id')

In [73]:
df_report

,trx_vel_last_2mths,amt_vel_last_1mths,card_present,cod_month,trx_vel_last_1mths,transaction_id,prediction,amt_vel_last_2mths,is_fraud
0,10,640.6695,1,202501,4,65930a764da2,0.0001422771019861102,1391.082,0
1,7,450.68726,1,202501,3,65faa84a083c,0.0001422771019861102,983.5444,0


In [ ]:
loss = log_loss(y_true, y_pred)

In [83]:
loss = log_loss(df_report['is_fraud'], df_report['prediction'], labels=[0, 1])
loss

0.00014228722433311518